----------

https://github.com/meierale/hatebase-api-python/blob/master/hatebase_api_howto.ipynb

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
# initialize variables
auth_url = "https://api.hatebase.org/4-4/authenticate"
vocab_url = "https://api.hatebase.org/4-4/get_vocabulary"
lang = "eng"
resp_format = "json"
token = ""
pages = 0
total_entries = 0

In [ ]:
# initialize authentication payload
auth_payload = "api_key=trjXqgqGxcokuCprecm8oZohfykbydbM"
headers = {
    'Content-Type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache"
    }

In [ ]:
# authenticate
auth_resp = requests.request("POST", auth_url, data=auth_payload, headers=headers)

In [ ]:
# get the session token
token = auth_resp.json()["result"]["token"]

In [ ]:
# quick check if it worked
print(auth_resp.text, token)

In [ ]:
# initialize vocabulary payload
# first without any given page-number
vocab_payload = "token=" + token + "&format=" + resp_format + "&language=" + lang
voc_resp = requests.request("POST", vocab_url, data=vocab_payload, headers=headers)
voc_json = voc_resp.json()

In [ ]:
# how many pages in total? and how many results= entries in the vocab?
pages = voc_json["number_of_pages"]
results = voc_json["number_of_results"]

# check pages & results
pages, results

In [ ]:
# create vocabulary df from first resultset
df_voc = pd.DataFrame(voc_json["result"])

In [ ]:
# now get results of all remaining pages and append to df_voc
for page in range(2,pages+1):
    vocab_payload = "token=" + token + "&format=json&language=" + lang + "&page=" + str(page)
    voc_resp = requests.request("POST", vocab_url, data=vocab_payload, headers=headers)
    voc_json = voc_resp.json()
    df_voc = df_voc.append(voc_json["result"])

# reset df_voc index
df_voc.reset_index(drop=True, inplace=True)

In [ ]:
df_voc.shape

In [ ]:
df_voc.to_csv("hatebase_vocab.csv")

------

In [3]:
## 1. Removal of punctuation and capitlization
## 2. Tokenizing
## 3. Removal of stopwords
## 4. Stemming

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import nltk

stopwords = nltk.corpus.stopwords.words("english")

#extending the stopwords to include other words used in twitter such as retweet(rt) etc.
other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)
stemmer = PorterStemmer()

def preprocess(tweet):  
    
    # removal of extra spaces
    regex_pat = re.compile(r'\s+')
    tweet_space = tweet.str.replace(regex_pat, ' ')

    # removal of @name[mention]
    regex_pat = re.compile(r'@[\w\-]+')
    tweet_name = tweet_space.str.replace(regex_pat, '')

    # removal of links[https://abc.com]
    giant_url_regex =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = tweet_name.str.replace(giant_url_regex, '')
    
    # removal of punctuations and numbers
    punc_remove = tweets.str.replace("[^a-zA-Z]", " ")
    # remove whitespace with a single space
    newtweet=punc_remove.str.replace(r'\s+', ' ')
    # remove leading and trailing whitespace
    newtweet=newtweet.str.replace(r'^\s+|\s+?$','')
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')
    # removal of capitalization
    tweet_lower = newtweet.str.lower()
    
    # tokenizing
    tokenized_tweet = tweet_lower.apply(lambda x: x.split())
    
    # removal of stopwords
    tokenized_tweet=  tokenized_tweet.apply(lambda x: [item for item in x if item not in stopwords])
    
    # stemming of the tweets
    tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 
    
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
        tweets_p= tokenized_tweet
    
    return tweets_p

In [4]:
import pandas as pd
lexicon = pd.read_csv('hatebase_vocab.csv')

terms = lexicon.term
lexicon['term'] = preprocess(terms)

In [6]:
lexicon.to_csv('hatebase_vocab.csv', index=False)